In [1]:
import numpy as np
from pydgrid import grid

In [95]:
Z_red = 110.0e3**2/5000.0e6
ratio=0.1
# Z**2 = R**2 + X**2
# Z**2 = ratio**2*X**2 + X**2 = X**2 * (ratio**2 + 1.0)
X_red = (Z_red**2/(ratio**2 + 1.0))**0.5
R_red = ratio*X_red

Z_trafo_b = 20.0e3**2/25.0e6
V_hv = 110.0/np.sqrt(3)*1.0

S1,pf1,S2,pf2 = 15300.0,0.98,5280,0.95
S_12_cplx = S1*pf1 + 1j*np.sin(np.arccos(pf1))*S1 + S2*pf2 + 1j*np.sin(np.arccos(pf2))*S2
pf_12 = np.cos(np.angle(S_12_cplx))

S1,pf1 = 40.0,0.85
S_13_cplx = S1*pf1 + 1j*np.sin(np.arccos(pf1))*S1
pf_13 = np.cos(np.angle(S_13_cplx))

S1,pf1,S2,pf2 = 215.0,0.97,390.0,0.85
S_14_cplx = S1*pf1 + 1j*np.sin(np.arccos(pf1))*S1 + S2*pf2 + 1j*np.sin(np.arccos(pf2))*S2
pf_14 = np.cos(np.angle(S_14_cplx))

data = {     
        "buses":[
                 {"bus": "HVinf", "pos_x":   0, "pos_y":  0, "units": "m", "U_kV":110.0},
                 {"bus": "HV",    "pos_x":   0, "pos_y": -5, "units": "m", "U_kV":110.0},
                 {"bus": "12",    "pos_x":   0, "pos_y":-15, "units": "m", "U_kV":20.0},
                 {"bus": "13",    "pos_x":   0, "pos_y":-35, "units": "m", "U_kV":20.0},
                 {"bus": "14",    "pos_x":   0, "pos_y":-85, "units": "m", "U_kV":20.0},
                ],
        "grid_formers":[
                        {"bus": "HVinf",
                        "bus_nodes": [1, 2, 3], "deg": [0, -120, -240],
                        "kV": [V_hv,V_hv,V_hv]}
                       ],
        "transformers":[
                        {"bus_j": "HV",  "bus_k": "12",  "S_n_kVA": 25.0e3, "U_j_kV":110, "U_k_kV":20.0,
                         "R_cc_pu": 0.001, "X_cc_pu":0.12, "connection": "Dyn5",   "conductors_j": 3, "conductors_k": 4}
                       ],
        "lines":[
                {"bus_j": "HVinf",  "bus_k": "HV",  "code": "eq_red", "m": 1000.0},
                {"bus_j": "12",     "bus_k": "13",  "code": "1", "m": 4.89e3},
                {"bus_j": "13",     "bus_k": "14",  "code": "1", "m": 3.0e3},
                ],
        "shunts":[
                 {"bus": "12" , "R": 0.001,   "X": 0.0, "bus_nodes": [4,0]},
                 {"bus": "13" , "R": 1.0e-10, "X": 0.0, "bus_nodes": [4,0]},
                 {"bus": "14" , "R": 1.0e-10, "X": 0.0, "bus_nodes": [4,0]},
                 ],
        "loads":[
                 {"bus": "12" , "kVA":S_12_cplx, "pf": pf_12,"type":"3P+N"},
                 {"bus": "13" , "kVA":40.0, "pf": 0.85,"type":"3P+N"},
                 {"bus": "14" , "kVA":S_14_cplx, "pf":pf_14,"type":"3P+N"},
                ],
        "line_codes":
            {
            "eq_red":  {"R1":R_red,"X1":X_red},    
            "1":  {"R1":0.510,"X1":0.366,"B1_mu":3.172,"R0":0.658,"X0":1.611,"B0_mu":1.28},
            "2":  {"R1":89.0e-3,"X1":3.5e-3},
            }
       } 

In [96]:
sys1 = grid()
sys1.read(data)  # Load data
sys1.pf()  # solve power flow
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

from pydgrid.plot_bokeh import plot_results
fig = plot_results(sys1)

In [90]:
Z_trafo = 0.016+1j*1.92
I_cc_12 = 20.0e3/np.sqrt(3)/Z_trafo
Z_base_trafo = 20.0e3**2/25.0e6
Z_trafo_pu = Z_trafo/Z_base_trafo

In [44]:
abs(I_cc_12)

6013.856493222156

In [40]:
Z_trafo_pu

(0.001+0.12j)

In [62]:
sys1.lines

[{'bus_j': 'HVinf',
  'bus_k': 'HV',
  'code': 'eq_red',
  'm': 1000.0,
  'type': 'z',
  'bus_j_nodes': [1, 2, 3],
  'bus_k_nodes': [1, 2, 3],
  'i_j_a_m': 22727.22626214114,
  'i_j_b_m': 22727.319192311817,
  'i_j_c_m': 0.05254491011279482,
  'i_j_n_m': 0.04084834980816039,
  'deg_j_a': -54.2893951525259,
  'deg_j_b': 125.71058142992896,
  'deg_j_c': -59.995823755827736,
  'deg_j_n': 120.00032122126828,
  'i_k_a_m': 22727.22626214114,
  'i_k_b_m': 22727.319192311817,
  'i_k_c_m': 0.05254491011279482,
  'i_k_n_m': 0.04084834980816039,
  'deg_k_a': -54.2893951525259,
  'deg_k_b': 125.71058142992896,
  'deg_k_c': -59.995823755827736,
  'deg_k_n': 120.00032122126828,
  'i_z': 0.01361611660272013,
  'i_p': 13121.593737360612,
  'i_n': 13121.600316741906},
 {'bus_j': '12',
  'bus_k': '13',
  'code': '1',
  'm': 4890.0,
  'type': 'pi',
  'bus_j_nodes': [1, 2, 3],
  'bus_k_nodes': [1, 2, 3],
  'i_j_a_m': 0.25027874847501763,
  'i_j_b_m': 6.058594898706528e-12,
  'i_j_c_m': 0.2502787484712338,

In [37]:
5774.2345111669765*1.1

6351.657962283674

In [49]:
5000*1.1

5500.0